In [1]:
import os
import warnings
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

from scipy.signal import savgol_filter
from scipy.interpolate import *
from scipy.optimize import curve_fit, least_squares

mpl.rc_file_defaults()
mpl.rc('text', usetex = True)
mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
params = {'text.latex.preamble': [
    r'\usepackage{sfmath}',
    r'\usepackage{helvet}',
    r'\usepackage{siunitx}',
    r'\sisetup{detect-family = true}',
    r'\usepackage{amsmath}'
]}
plt.rcParams['figure.max_open_warning'] = 100
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.axisbelow'] = True
if 'nbagg.transparent' in plt.rcParams.keys():
    plt.rcParams['nbagg.transparent'] = False
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['axes.grid'] = True
plt.rc('grid', linestyle='--', linewidth=0.5)
plt.rc('figure', figsize=(3, 2.5), facecolor='white')
plt.rc('savefig', dpi=600, bbox='tight', facecolor='white')
plt.rcParams.update(params)

warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

C:\Users\kpebe\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)
C:\Users\kpebe\AppData\Local\Continuum\anaconda3\lib\site-packages\matplotlib\__init__.py:850: MatplotlibDeprecationWarning: 
The nbagg.transparent rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1. Use figure.facecolor instead.
  version, name=key, obj_type="rcparam", alternative=alt_key)


In [2]:
%matplotlib notebook

In [17]:
def check_file(df, rmin=110e6, vgmax=10, vgmin=-10):
    df['id'] = (df['id1'] + df['id2']) / 2
    df['ig'] = (df['ig1'] + df['ig2']) / 2
    if df.isnull().values.any() or len(df) < 1 or (df.values > 1e10).any():
        # No data
        return None, None, 'nodata'
    
    cur_filtered = savgol_filter(df['id'], window_length=13, polyorder=2)
    df['sgf'] = cur_filtered
    
    if np.sum((cur_filtered - df['id']) ** 2) > len(df)*5e-15:
        # Noisy
        return None, None, 'noisy'

    rg = np.diff(df['vg'])/np.diff(df['ig'])
    pos_rg = rg[rg > 0]
    
    p = len(pos_rg[pos_rg < rmin]) / len(pos_rg)
    imin = np.average(df['id'][df['vg'] > vgmax])
    imax = np.average(df['id'][df['vg'] < vgmin])
    
    if df['vb'][0] / imax > rmin or imax < 0 or imin < 0:
        # Nonconductive
        return p, None, 'noncond'
    
    return p, imax/imin, 'ok'


def get_mobility(df, lc, wc):
    c = 3.9 * 8.85e-12 / 300e-9 
    df['mu'] = df['sgf'].diff()/df['vg'].diff() *lc/(wc*df['vb'][0]*c)*1e4
    return df.loc[df['vg'].abs() < 2.5, 'mu'].abs().mean()

In [25]:
def fill(directory, offset, plot=False):
    pmax = 0.05
    res = ['txt', 'dat', 'csv']
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.split('.')[-1] in res:
                df = pd.read_csv((os.path.join(root, file)), sep='\t', decimal='.',\
                    skiprows=2, header=None, names=['vb', 'vg', 'id1', 'ig1', 'id2', 'ig2'])
                df
                coords = np.array(file.split('.')[-2].split('_')[-2:], 'int') + offset
                y, x = coords[0], coords[1]
                p, onoff, status = check_file(df)
                if status == 'nodata':
                    # No data - state=0
                    state.loc[y,x] = 0
                elif status == 'noisy':
                    # Noisy - state=1
                    if plot:
                        fig, ax = plt.subplots()
                        ax.plot(df['vg'], df['id'], label='data')
                        ax.plot(df['vg'], df['sgf'], label='smoothed')
                        ax.set_title('x = '+str(x)+', y = '+str(y))
                        ax.legend();
                    state.loc[y,x] = 1
                elif status == 'noncond':
                    # Nonconductive - state=2
                    state.loc[y,x] = 2
                else:
                    if p <= pmax:
                        if (onoff is not None) and type(onoff) != str:
                            lc = 50 + 50*(x-1) + 25*((numy-y) // (numy//2))
                            mobi =  get_mobility(df, lc, 150)
                            if mobi > 350 and plot:
                                fig, ax = plt.subplots()
                                ax.plot(df['vg'], df['id'], label='mu = '+'{:.2g}'.format(mobi))
                                ax.plot(df['vg'], df['sgf'])
                                ax.set_title('x = '+str(x)+', y = '+str(y))
                                ax.legend();
                            mobilities.loc[y,x] = mobi
                            ratios.loc[y,x] = onoff
                            if onoff > 100 and plot:
                                fig, ax = plt.subplots()
                                ax.plot(df['vg'], df['id'], label='onoff = '+'{:.2g}'.format(onoff))
                                ax.set_yscale('log')
                                ax.set_title('x = '+str(x)+', y = '+str(y))
                                ax.legend();
                    else:
                        # Leak - state=3
                        state.loc[y,x] = 3                    

In [26]:
numx,numy = 4,12

ratios = pd.DataFrame(np.nan, index=np.linspace(1,numy,numy), columns=np.linspace(1,numx,numx))
state = ratios.copy()
mobilities = ratios.copy()

directory =  r'C:\Users\kpebe\OneDrive\Рабочий стол\Anton\./dat'
fill(directory, offset=np.array([0, 0]), plot=True)

In [24]:
fig, ax = plt.subplots(1, 2, figsize=(8,3.3))
auxmap = mpl.colors.ListedColormap(['mediumseagreen', 'darkturquoise', 'royalblue', 'magenta'])
ratios.fillna(value=np.nan, inplace=True)
x, y = np.arange(ratios.shape[1]+1), np.arange(ratios.shape[0]+1)

im = ax[0].pcolormesh(x, y, ratios, cmap=mpl.cm.YlOrBr,\
                   norm=mpl.colors.LogNorm())
ax[0].pcolormesh(x, y, state, vmin=0, vmax=3, cmap=auxmap)

ax[0].set_ylim(ax[0].get_ylim()[::-1])
ax[0].xaxis.tick_top()
divider = make_axes_locatable(ax[0])
cax = divider.append_axes("right", "5%", pad="3%")
cb = fig.colorbar(im, ax=ax[0], cax=cax, orientation='vertical', aspect=40)
cb.set_label(r'I\textsubscript{on}/I\textsubscript{off}')

im = ax[1].pcolormesh(x, y, mobilities, cmap=mpl.cm.YlOrBr,\
                   norm=mpl.colors.LogNorm())
ax[1].pcolormesh(x, y, state, vmin=0, vmax=3, cmap=auxmap)

ax[1].set_ylim(ax[1].get_ylim()[::-1])
ax[1].xaxis.tick_top()
divider = make_axes_locatable(ax[1])
cax = divider.append_axes("right", "5%", pad="3%")
cb = fig.colorbar(im, ax=ax[1], cax=cax, orientation='vertical', aspect=40)
cb.set_label(r'\textmu,\,\si{\cm^2\volt^{-1}\s^{-1}}')

patches = [mpl.patches.Patch(color='mediumseagreen', label='No data'),
           mpl.patches.Patch(color='darkturquoise', label='Noisy data'),
           mpl.patches.Patch(color='royalblue', label='Non conductive'),
           mpl.patches.Patch(color='magenta', label='Gate leakage')]
lgd = fig.legend(handles=patches, loc=9, ncol=4)
fig.tight_layout(pad=2.7, w_pad=0.3)
fig.savefig("All_party.png", dpi=600, bbox_extra_artists=(lgd,), bbox_inches='tight');

<IPython.core.display.Javascript object>

FileNotFoundError: [WinError 2] Не удается найти указанный файл

In [12]:
p

NameError: name 'p' is not defined